# Um Modelo de Recomendação da Movie Lens

## Introdução

O dataset [MovieLens](https://grouplens.org/datasets/movielens/) é um conjunto de dados que contém várias classificações (ratings) de filmes por usuários. Este conjunto de dados é frequentemente usado para experimentos de pesquisa e prototipagem em sistemas de recomendação. O projeto MovieLens foi criado pelo GroupLens, um laboratório de pesquisa na University of Minnesota, e serve como uma das referências padrão na área de sistemas de recomendação.

O conjunto de dados tem várias versões, algumas contendo apenas algumas dezenas de milhares de classificações e outras contendo até 20 milhões ou mais. Cada registro geralmente contém:

    ID do usuário que deu a classificação
    ID do filme classificado
    A classificação dada (geralmente em uma escala de 1 a 5)
    Um atributo timestamp indicando quando a classificação foi dada

Algumas versões também incluem informações adicionais, como tags atribuídas aos filmes, gêneros, e até mesmo links para dados relacionados, como imagens de capas de filmes ou metadados. Por ser um conjunto de dados bem estruturado e extensivo, o MovieLens é amplamente utilizado para demonstrar algoritmos de recomendação, desde métodos simples, como a Filtragem Colaborativa e a Filtragem Baseada em Conteúdo, até técnicas mais avançadas como Sistemas de Recomendação baseados em Aprendizado Profundo.

## Baixando o dataset

In [ ]:
# No Colab não é necessário executar esse comando
# %pip install gdown

In [ ]:
# Importa o dataset de filmes
!gdown 1ovr90WWjeLh_PWqe5yLIK-1aIzHvtuDk

In [ ]:
# Import o dataset com as avaliações dos usuários
!gdown 1pN2Upg6J7mie1esMREFD5rHjZOpk9QXb

## Preparando o dataset para o treinamento

**Bibliotecas**

In [3]:
import pandas as pd
import numpy as np

### `movies.dat`

**Testando o acesso ao dataset**

In [4]:
# Adapte o nome do caminho para usar o caderno no Colab
df_filmes = pd.read_csv('./movies.dat', sep='::', engine='python', names=['id_filme', 'nome', 'categoria'])
df_filmes.head()

,id_filme,nome,categoria
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
print(df_filmes.id_filme.nunique())

10681


### `ratings.csv`

**Testando o acesso ao dataset de ratings**

In [6]:
#  Adapte o nome do caminho para usar o caderno no Colab
df_ava = pd.read_csv(
        './ratings_mini.csv',
        sep= ',', # Separador
        dtype={'id_usuario': np.int32, 'id_filme': np.int32, 'avaliacao': np.float64, 'timestamp':np.int32})
df_ava.head()

,id_usuario,id_filme,avaliacao,timestamp
0,5,1,1.0,857911264
1,5,7,3.0,857911357
2,5,25,3.0,857911265
3,5,28,3.0,857913507
4,5,30,5.0,857911752


In [7]:
print(df_ava.id_usuario.nunique())

# Há filmes que não foram avaliados
print(df_ava.id_filme.nunique())

33875
10010


In [8]:
print(len(df_ava))

4866210


## Treinando o modelo de recomendação

### Filtragem Colaborativa

A filtragem colaborativa faz recomendações com base em padrões de comportamento passado de vários usuários, sem necessitar de qualquer informação adicional sobre os itens ou usuários. A ideia básica da filtragem colaborativa é criar uma matriz usuário-item. O conjunto de dados pode ser representado como uma matriz onde as linhas correspondem aos usuários e as colunas aos filmes.


 Matrix | Inception | Titanic | Star Wars | The Godfather
--------|-----------|---------|-----------|--------------
Alice    |     5     |    3    |     4     |      0
Bob      |     4     |    0    |     5     |      3
Carol    |     3     |    5    |     4     |      4
Dave     |     0     |    2    |     0     |      5
Eve      |     2     |    5    |     0     |      4


O `scikit-surprise` converte esse formato longo em uma matriz usuário-item internamente para executar algoritmos como Single Value Decomposition (SVD).

In [9]:
%pip install surprise

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [10]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

#### Treinamento do modelo de SVD

In [29]:
# As avaliações vão de 1 a 5 no dataset
reader = Reader(rating_scale=(1, 5))

# Carrega os dados para um formato que o SVD consegue ler
data = Dataset.load_from_df(df_ava[['id_usuario', 'id_filme', 'avaliacao']], reader)

# Cria um modelo SVD
model = SVD()

# Realizando validação cruzada
resultados_cv = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Construindo conjunto de treinamento
trainset = data.build_full_trainset()

# Ajusta o modelo SVD
model.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8263  0.8267  0.8244  0.8271  0.8262  0.8262  0.0009  
MAE (testset)     0.6353  0.6353  0.6336  0.6361  0.6352  0.6351  0.0008  
Fit time          62.32   70.03   65.53   79.58   79.29   71.35   7.04    
Test time         18.99   15.68   14.10   29.67   15.02   18.69   5.73    


Exemplo de uso.

In [30]:
user_id = 7
filme_id = 5 

model.predict(user_id, filme_id)

Prediction(uid=7, iid=5, r_ui=None, est=2.5322582793130977, details={'was_impossible': False})

Intepretando esse resultado, temos que:

* `uid`: ID do usuário

* `iid`: ID do item (no nosso caso, filmes)

* `r_ui`: Avaliação real data pelo usuário ao item. O valor `None` sugere que o usuário não avaliou esse item

* `est`: Previsão da avaliação do usuário.

* `details={'was_impossible': False}`: Este campo fornece informações adicionais sobre a previsão. O campo `was_impossible` indica se a previsão poderia ser feita ou não. Neste caso, é False, o que significa que a previsão foi possível.

#### Persistindo o modelo

In [ ]:
from joblib import dump
dump(model, './movie_recommender_model.joblib')